In [1]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install pymysql

In [2]:
# !pip install webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pymysql
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [36]:
# 데이터 확인용
one = []  #기업명
two = []  #근무지
three = [] #경력
four = [] #채용공고
five = [] #태그
six = [] #마감일
seven = [] #근무지(상세)


# df 데이터 변수
title = []           #채용공고
name = []            #기업명
career = []          #경력
loc = []             #근무지
tag = []             #태그
timeout = []         #마감일
loc2 = []            #근무지(상세)

# 스크롤 이동변수
current_scroll = 0
down_scroll = 0









# 처음 크롬 실행+메인화면 선택 

In [34]:
driver = webdriver.Chrome("")
time.sleep(5)
url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all'
driver.get(url)
time.sleep(5)












# 데이터 수집

In [63]:
def craw():
    for j in range(1,5):
        time.sleep(3)
        if j == 1:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobHeader_JobHeader__DBrFJ')
                
                name.append(element.text.split('\n')[0].split('∙')[0])
                loc.append(element.text.split('\n')[0].split('∙')[1])
                career.append(element.text.split('\n')[0].split('∙')[2])
                title.append(element.text.split('\n')[1])
            except:
                name.append('')
                loc.append('')
                career.append('')
                title.append('')
        elif j == 2:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'CompanyTags_CompanyTags__wpSCo')
                if len(element.text.split('\n')) >=1:
                    tag.append(element.text.split('\n')[1:])
                else:
                    tag.append('')    
            except:
                tag.append('')
        elif j == 3:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobDueTime_JobDueTime__iKbEO')
                if len(element.text.split('\n')) ==2:
                    timeout.append(element.text.split('\n')[1])
                else:
                    timeout.append('')
            except:
                timeout.append('')
        else:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobWorkPlace_JobWorkPlace__5fs5x')
                if len(element.text.split('\n')) == 2:
                    loc2.append(element.text.split('\n')[1])
                else:
                    loc2.append('')    
            except:
                loc2.append('')

In [62]:
def page_down():
    global done
    while True:
        current_scroll = driver.execute_script('return window.pageYOffset')
        driver.execute_script(f'window.scrollTo(0, {current_scroll - 50});')
        time.sleep(1)
        driver.execute_script(f'window.scrollTo(0, {current_scroll + 150});')
        time.sleep(1)
        down_scroll = driver.execute_script('return window.pageYOffset')
        if current_scroll == down_scroll:
            done = True
            break
        try:
            driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{i}]/div/a/div[1]').click()
            driver.switch_to.window(driver.window_handles[0])
            craw()
            driver.back()
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[0])  
            break
        except:
            return page_down()
    return









# 메인 실행

In [67]:
title = []           #채용공고
name = []            #기업명
career = []          #경력
loc = []             #근무지
tag = []             #태그
timeout = []         #마감일
loc2 = []            #근무지(상세)

done = False

# 스크롤 이동변수
current_scroll = 0
down_scroll = 0


driver = webdriver.Chrome("")
time.sleep(5)
url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all'
driver.get(url)

for i in tqdm(range(1,30)):
    try:
        if (i % 50) == 0:
            driver.close()
            time.sleep(5)
            driver = webdriver.Chrome("")
            time.sleep(5)
            url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all'
            driver.get(url)
            time.sleep(5)
        try:
            time.sleep(5)
            driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{i}]/div/a/div[1]').click()
            driver.switch_to.window(driver.window_handles[0])
            craw()
            driver.back()
            driver.switch_to.window(driver.window_handles[0])                        
        except:
            page_down()
        if done == True:
            print(f'{i}개의 데이터 저장')
            break
    except:
        driver.close()
        time.sleep(2)
        driver = webdriver.Chrome("")
        time.sleep(5)
        url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all'
        driver.get(url)
        time.sleep(5)
        if (i % 100) == 0:
            driver.close()
            time.sleep(2)
            driver = webdriver.Chrome("")
            time.sleep(5)
            url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all'
            driver.get(url)
            time.sleep(5)
        try:
            time.sleep(5)
            driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{i}]/div/a/div[1]').click()
            driver.switch_to.window(driver.window_handles[0])
            craw()
            driver.back()
            driver.switch_to.window(driver.window_handles[0])                        
        except:
            page_down()
        if done == True:
            print(f'{i}개의 데이터 저장')
            break

  0%|                                                                | 293/999999998 [2:03:34<7029762:28:37, 25.31s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.0)
Stacktrace:
	GetHandleVerifier [0x00007FF6313D3432+62578]
	(No symbol) [0x00007FF63134C589]
	(No symbol) [0x00007FF631207DCA]
	(No symbol) [0x00007FF6311DD5B5]
	(No symbol) [0x00007FF631283407]
	(No symbol) [0x00007FF63128A562]
	(No symbol) [0x00007FF63127BF30]
	(No symbol) [0x00007FF63124934E]
	(No symbol) [0x00007FF631249F41]
	GetHandleVerifier [0x00007FF6316E99CD+3299853]
	GetHandleVerifier [0x00007FF6317358D0+3610896]
	GetHandleVerifier [0x00007FF63172B930+3570032]
	GetHandleVerifier [0x00007FF631483376+783286]
	(No symbol) [0x00007FF63135763F]
	(No symbol) [0x00007FF631352A14]
	(No symbol) [0x00007FF631352BA2]
	(No symbol) [0x00007FF63134270F]
	BaseThreadInitThunk [0x00007FFF5B137344+20]
	RtlUserThreadStart [0x00007FFF5C3C26B1+33]


In [55]:
driver = webdriver.Chrome("")
time.sleep(5)
url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all'
driver.get(url)
time.sleep(5)

In [68]:
print(len(name))
print(len(title))
print(len(loc))
print(len(loc2))
print(len(tag))
print(len(timeout))
print(len(career))


# print(name)
print(title)
# print(loc)
# print(loc2)
# print(tag)
# print(timeout)
# print(career)

293
293
293
293
293
293
293
['Front-end 개발자', 'AI 엔지니어 (1년 이상)', 'Dev Ops 개발자(3년 이상)', '딥러닝 엔지니어', 'AI 엔지니어', 'AI Research Leader', '백엔드 개발자', '보안솔루션 기술지원 엔지니어 (1년 이상)', '음성 및 자연어 AI Researcher 및 AI Senior Researcher', '[서울] AI Researcher (언어모델, LLM, NLP)', 'Deep Learning Engineer', 'ML Researcher', '[TADA] DevOps Engineer', '애널리틱스 엔지니어', '머신러닝연구원(ML Researcher)', '신약개발 인공지능 연구원 (3년이상)', '금융 데이터 솔루션(QUANDA) 개발자', '[그루비 SaaS 솔루션] 데이터사이언스팀 리더', '[1100억↑투자] Data Engineer (3년 이상)', 'FE 개발자', 'Software Test Automation Engineer', 'Senior Data Scientist (이미지 데이터)_5년 이상', 'AI Researcher (이미지 생성 모델링) 리더급', 'Data Scientist (3년~)', 'DW Engineer / 병원정보시스템 개발자', 'DA/DS 분석가', 'AI / 딥러닝 개발자', 'Senior Applicatin Engineer', 'Front-end Tech Leader', '[서울] AI Researcher (언어모델, LLM, NLP) (병역특례한정)', 'Technical Consultant', 'Data Architect (DA) (Data Ops팀)', '가상자산거래소 월렛팀 엔지니어', '[100억↑투자] 머신러닝 엔지니어(2년이상)', 'AI 응용 프로그램 개발 풀스택 개발자', 'Technical Project Manager (Data)', 'AI NLP 엔지니어(RAG 최적화)', '[100억↑투자] Data E

In [35]:
element = driver.find_element(By.CLASS_NAME, value = 'CompanyTags_CompanyTags__wpSCo')
element

<selenium.webdriver.remote.webelement.WebElement (session="6137ca60d71ab4b3074ba8128519f9ac", element="f.8856EEEC33D2746D000103E8DAE89E8F.d.2B0A0F1638D9903B352F13263FC509AA.e.162")>

In [14]:
driver = webdriver.Chrome("")
time.sleep(5)
url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all'
driver.get(url)

In [19]:
current_scroll = driver.execute_script('return window.pageYOffset')

In [20]:
driver.execute_script(f'window.scrollTo(0, {current_scroll + 100});')

In [ ]:
down_scroll = driver.execute_script('return window.pageYOffset')

In [73]:
tag_str = []

In [74]:
df = pd.DataFrame()
for i in tag:
     tag_str.append(','.join(i))

df['기업명'] = name
df['채용공고'] = title
df['위치'] = loc
df['위치상세'] = loc2
df['태그'] = tag_str
df['마감일'] = timeout
df['경력'] = career

df = df.drop_duplicates()
df = df.fillna('')
df.to_csv(f'C:\\Users\\EZEN\pro\\pro_data\\wanted_engineer.csv',encoding='utf-8')





,기업명,채용공고,위치,위치상세,태그,마감일,경력
0,덴컴,Front-end 개발자,서울 강남구,"서울특별시 강남구 삼성로85길 26, V&S 14층","유연근무,리프레시휴가,편의시설,식대지원,유망산업,스톡옵션,커피·스낵바,설립4~9년,...",상시채용,경력 4-10년
1,한국딥러닝주식회사,AI 엔지니어 (1년 이상),서울 서초구,서초구 강남대로89길 30,"편의시설,보너스,연봉상위2~5%,50명이하,설립4~9년,유망산업,유연근무,리프레시휴...",상시채용,경력 1-10년
2,휴톰,Dev Ops 개발자(3년 이상),서울 마포구,서울특별시 마포구 독막로 279 상장회사회관 6층,"유망산업,건강검진지원,커피·스낵바,복지포인트,보너스,유연근무,장비지원,자유로운휴가,...",상시채용,경력 3-20년
3,펀진,딥러닝 엔지니어,서울 성동구,서울시 성동구 연무장5가길 7 현대테라스타워 East동 13층,"편의시설,스톡옵션,보너스,육아휴직,설립10년이상,유연근무,리프레시휴가,편의시설,식대...",상시채용,경력 1-3년
4,유니드컴즈,AI 엔지니어,서울 마포구,"서울 마포구 양화로 81, 패스트파이브 합정점 L1 L105호 유니드컴즈","자기계발지원,재택근무,식대지원,유망산업,설립10년이상,유연근무,리프레시휴가,편의시설...",상시채용,경력 3-10년
...,...,...,...,...,...,...,...
288,한국신용데이터(KCD),[2600억↑투자] [데이터전략실] Data Engineer (DW & Mart),서울 강남구,"서울특별시 강남구 테헤란로 127, 4층(하나금융그룹 강남사옥)","자기계발지원,식대지원,커피·스낵바,보너스,건강검진지원,장비지원,의료비지원,스톡옵션,...",상시채용,경력 3-10년
289,비바리퍼블리카(토스),Data Analytics Engineer,서울 강남구,테헤란로 142 아크플레이스 12층,"단체보험,통신비,재택근무,커피·스낵바,편의시설,보너스,301~1,000명,설립10년이상",상시채용,경력 2-10년
290,카카오뱅크(kakaobank),데이터웨어하우스 운영 담당자,경기 성남시,"(도로명) 경기도 성남시 분당구 분당내곡로 131 판교테크원타워, 타워2 11층","편의시설,의료비지원,건강검진지원,어린이돌봄,휴가비,자기계발지원,유연근무,인원급성장,...",상시채용,경력 3-10년
291,스카이랩스(SkyLabs),[400억↑투자] 딥러닝/머신러닝(AI) 개발자 (신입/병역특례 전문연구요원 가능),경기 성남시,"성남시 분당구 판교로 255번길 58, 씨즈타워 703호","자율복장,편의시설,식대지원,수평적조직,커피·스낵바,유망산업,유연근무,전문연구요원,설...",상시채용,신입


In [76]:
conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
cur = conn.cursor()

In [77]:
cur.execute('''CREATE TABLE IF NOT EXISTS pro_wanted_engineer (
               name VARCHAR(255),
               title VARCHAR(255),
               time VARCHAR(255),
               loc VARCHAR(255),
               career VARCHAR(255),
               info VARCHAR(255),
               loc2 VARCHAR(255)
            )''')

0

In [78]:
for index, row in df.iterrows():
    cur.execute(f"""
        INSERT INTO pro_wanted_engineer (
           name,
           title,
           time,
           loc,
           career,
           info,
           loc2
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        row['기업명'],
        row['채용공고'],
        row['마감일'],
        row['위치'],
        row['경력'],
        row['태그'],
        row['위치상세']
    ))  

In [79]:
# 변경사항 커밋
conn.commit()

# 연결 종료
conn.close()

In [85]:
url_list = ['https://www.wanted.co.kr/wdlist/518/873?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/669?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/899?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/1634?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/10231?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/1022?country=kr&job_sort=job.recommend_order&years=-1&locations=all']

url_name_list = ['wanted_web_developer',
                 'wanted_frontend_developer',
                 'wanted_python_developer',
                 'wanted_ML_engineer',
                 'wanted_data_engineer',
                 'wanted_data_scientist',
                 'wanted_DBA',
                 'wanted_BI_engineer']

In [88]:
def DB_insert(file_name):
    df = pd.read_csv('경로',encoding='utf-8')
    conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
    cur = conn.cursor()
    cur.execute(f'''CREATE TABLE IF NOT EXISTS pro_{file_name} (
                   name VARCHAR(255),
                   title VARCHAR(255),
                   time VARCHAR(255),
                   loc VARCHAR(255),
                   career VARCHAR(255),
                   info VARCHAR(255),
                   loc2 VARCHAR(255)
                )''')
    
    for index, row in df.iterrows():
        cur.execute(f"""
            INSERT INTO pro_{file_name} (
               name,
               title,
               time,
               loc,
               career,
               info,
               loc2
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            row['기업명'],
            row['채용공고'],
            row['마감일'],
            row['위치'],
            row['경력'],
            row['태그'],
            row['위치상세']
        ))  

In [87]:
def DF_make(file_name):
    df = pd.DataFrame()
    for i in tag:
         tag_str.append(','.join(i))

    df['기업명'] = name
    df['채용공고'] = title
    df['위치'] = loc
    df['위치상세'] = loc2
    df['태그'] = tag_str
    df['마감일'] = timeout
    df['경력'] = career

    df = df.drop_duplicates()
    df = df.fillna('')
    df.to_csv(f'C:\\Users\\EZEN\pro\\pro_data\\pro_{file_name}.csv',encoding='utf-8')

